## 12.4 Wstrzykiwanie interfejsów

W tym przykładzie przyjrzymy się procesowi wstrzyknięć interfejsów. W dalszym ciągu posługujemy się abstrakcją komputera - tym razem nasz komputer będzie składał się tylko z cpu. `CPU` jest interfejsem, mamy również dwie klasy implementujące ten interfejs - `AMD` i `Intel`.

In [ ]:
interface CPU {
    fun name(): String
}

class Intel : CPU {
    override fun name(): String {
        return "intel"
    }
}

class AMD : CPU {
    override fun name(): String {
        return "amd"
    }
}

class Computer @Inject constructor(val cpu: CPU) {
    fun cpuName(): String{
        return cpu.name()
    }
}

klasy `AMD` i `Intel` posiadają kontruktor z adnotacją `@Inject`

In [ ]:
class Intel @Inject constructor() : CPU {
    override fun name(): String {
        return "intel"
    }
}

class AMD @Inject constructor() : CPU {
    override fun name(): String {
        return "amd"
    }
}

Teraz `Dagger2` wie jak stworzyć te dwie klasy, jednak w konstruktorze klasy `Computer` przyjmujemy argument o typie interfejsu `CPU`, więc musimy jescze wskazać w odpowiednim module sposób inicjalizacji `CPU`. potrzebujemy dwa moduły - dla każdego `CPU`.

In [ ]:
@Module
class AmdModule {
    @Provides
    fun provideCpu(cpu: AMD): CPU{
        return cpu
    }
}

Nasz `AmdModule` zawiera jedną metodę `providesCpu` z adnotacją `@Provides`. Metoda przyjmuje jeden argument typu `AMD` i zwraca obiekt typu `CPU`.

W interfejsie `ComputerComponent` jako argument adnotacji `@Component`, podejemy listę wszystkich modułów - możemy podać tylko **jeden** moduł klasy rozszerzającej interfejs `CPU`.

In [ ]:
@Component(modules = [AmdModule::class])
interface ComputerComponent {
    fun inject(mainActivity: MainActivity)
}

Główna aktywność pozostaje bez zmian od ostatniego przykładu

In [ ]:
class MainActivity : AppCompatActivity() {

    @Inject
    lateinit var computer: Computer

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val component = DaggerComputerComponent.create()
        component.inject(this)
        val textView = findViewById<TextView>(R.id.textview)
        textView.text = computer.cpuName()
    }
}

Jeżeli chcemy wykorzystać `IntelModule`

In [ ]:
@Module
class IntelModule {
    @Provides
    fun provideCpu(cpu: Intel): CPU {
        return cpu
    }
}

Musimy zamienić nazwę modułu w intefejsie `ComputerComponent`

In [ ]:
@Component(modules = [IntelModule::class])
interface ComputerComponent {
    fun inject(mainActivity: MainActivity)
}

### **`@Binds`**

Ponieważ w klasie `AmdModule` posiadamy jedną metodę, zwracającą przesłany argument, możemy zoptymalizować kod i uniknąć tworzenia jej instancji. W tym celu zmienimy klasę na klasę abstrakcyjną, oraz metodę na metodę abstrakcyjną (bez ciała).

In [ ]:
@Module
abstract class AmdModule {
    @Binds
    abstract fun bindCpu(cpu: AMD): CPU
}